## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Sun Sep  1 09:17:27 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             12W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **03.01.2024** with version **YOLOv8.0.196**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [9]:
!pip install --upgrade ray
!pip install --upgrade ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 21.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: ray
    Found existing installation: ray 2.24.0
    Uninstalling ray-2.24.0:
      Successfully uninstalled ray-2.24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 738.2 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 6.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.196
    Uninstalling ultralytics-8.0.196:
      Successfully uninstalled ultralytics-8.0.196


In [10]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5846.2/8062.4 GB disk)


In [12]:
from ultralytics import YOLO

from IPython.display import display, Image

## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

In [6]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="**********************")
project = rf.workspace("nishant-raghuwanshi-deep-learning").project("skin-diseases-i30ay-9igew")
version = project.version(1)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 1.4 MB/s eta 0:00:00eta 0:00:01
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to skin-diseases-1 in yolov8:: 100%|██████████| 3310/3310 [00:00<00:00, 7540.36it/s]


## Custom Training

In [13]:
!yolo task=detect mode=train model=yolov8m.yaml data=/kaggle/input/yaml-file/data.yaml epochs=80 imgsz=800 plots=False

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics YOLOv8.2.85 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/kaggle/input/yaml-file/data.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_rati

In [16]:
!ls {HOME}/runs/detect/train3/                                   

args.yaml					   results.csv
events.out.tfevents.1725182996.574ed42710b1.258.0  weights


## Validate Custom Model

In [18]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train3/weights/best.pt data=/kaggle/input/yaml-file/data.yaml

/kaggle/working
Ultralytics YOLOv8.2.85 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m summary (fused): 218 layers, 25,846,708 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /kaggle/working/skin-diseases-1/valid/labels.cache... 264 images, 
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 175, len(boxes) = 863. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        264        863      0.547      0.487      0.489      0.273
                  Acne         27        382      0.565       0.51       0.51      0.163
            Chickenpox         21         71       0.35      0.183      0.198     0.0744
                Eczema         36        114      0.614      0.808      0.758      0.441
             Monkeypox      

## Inference with Custom Model

In [19]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train3/weights/best.pt conf=0.25 source=/kaggle/working/skin-diseases-1/test/images save=True

/kaggle/working
Ultralytics YOLOv8.2.85 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m summary (fused): 218 layers, 25,846,708 parameters, 0 gradients, 78.7 GFLOPs

image 1/193 /kaggle/working/skin-diseases-1/test/images/3_12_jpg.rf.14ac5a71a5eeabf7bea48d677ee63bc8.jpg: 800x800 (no detections), 58.8ms
image 2/193 /kaggle/working/skin-diseases-1/test/images/6_70_jpg.rf.8e605413f7d1e726832a3efa1770c978.jpg: 800x800 7 Ringworms, 42.1ms
image 3/193 /kaggle/working/skin-diseases-1/test/images/7_3_jpg.rf.27d8e0dc3ee0ab9b4b11129eae27ec4e.jpg: 800x800 1 Ringworm, 42.2ms
image 4/193 /kaggle/working/skin-diseases-1/test/images/800px-Yeartinfection-Copy_jpg.rf.7ef412c370a258b6fc09f55f45863242.jpg: 800x800 1 Psoriasis, 42.2ms
image 5/193 /kaggle/working/skin-diseases-1/test/images/CHP_03_01_jpg.rf.755d105d37d998b2c851ce0f93adf1fa.jpg: 800x800 8 Chickenpoxs, 42.2ms
image 6/193 /kaggle/working/skin-diseases-1/test/images/CHP_06_01_jpg.rf.b2a40771dbf9f57bba2fa4470523a282.jpg: 800x80

**NOTE:** Let's take a look at few results.

## Deploy model on Roboflow

Once you have finished training your YOLOv8 model, you’ll have a set of trained weights ready for use. These weights will be in the `/runs/detect/train/weights/best.pt` folder of your project. You can upload your model weights to Roboflow Deploy to use your trained weights on our infinitely scalable infrastructure.

The `.deploy()` function in the [Roboflow pip package](https://docs.roboflow.com/python) now supports uploading YOLOv8 weights.

To upload model weights, add the following code to the “Inference with Custom Model” section in the aforementioned notebook:

In [20]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train3/")

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


View the status of your deployment at: https://app.roboflow.com/nishant-raghuwanshi-deep-learning/skin-diseases-i30ay-9igew/1
Share your model with the world at: https://universe.roboflow.com/nishant-raghuwanshi-deep-learning/skin-diseases-i30ay-9igew/model/1
